# HUXt Processing

<div style="background-color: #ffaaaa; padding: 15px; border-radius: 5px;">
<b>Important Note:</b> Must create HUXt dataframes from "ambient_huxt.ipynb" first.
</div>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

huxt_utils_dir = os.path.join(os.getcwd(), 'src', 'huxt')
ml_utils_dir = os.path.join(os.getcwd(), 'src', 'ml')
data_dir = os.path.join(os.getcwd(), 'src', 'data')

# Add my utils to the path
import sys
sys.path.append(huxt_utils_dir)
sys.path.append(ml_utils_dir)

from data_loader import load_huxt_data_as_windows, load_omni_data
import huxt_utils as HU
import fastparquet

In [ ]:
%%time 

# Specify which CRs 
start_cr = 1892   # Min 1892
end_cr = 2290     # Max 2290

additional_cols = ['hp30', 'velocity gradient']
folder_name = f'HUXt1'
print(f'Processing {folder_name}')
for cr in range(start_cr, end_cr):
    df = HU.huxt_output_to_ml_df(rotation_number=cr, extra_columns=additional_cols,
                                 folder_name=folder_name, save=True, overwrite=True)

In [ ]:
chunk_size = 20  # Process [chunk_size] files at a time
output_file = os.path.join(data_dir, 'HUXt', f'{folder_name}_modified', 'full_df.parquet')
huxt_data_dir = os.path.join(data_dir, 'HUXt', f'{folder_name}_modified')

dfs = []  
last_index = None  # Keep track of last index to remove overlap
Nens = 100         # Specify no. ensembles

OMNI = load_omni_data(data_dir)

def process_chunk():
    """Saves the current chunk of data, removing duplicates and appending to the Parquet file."""
    global dfs, last_index  # Ensure we modify the global list and last index
    
    if not dfs:  # If there's no data, skip saving
        return

    # Concatenate and remove duplicates
    chunk_df = pd.concat(dfs, ignore_index=False)

    # Drop duplicated indices while keeping the last occurrence
    chunk_df = chunk_df[~chunk_df.index.duplicated(keep='last')]

    # Remove overlap with the last processed chunk
    if last_index is not None:
        chunk_df = chunk_df.loc[chunk_df.index > last_index]

    # Update last index for next batch
    last_index = chunk_df.index[-1]
    
    OMNI_half_hourly = OMNI.reindex(chunk_df.index, method='ffill')

    v_columns = chunk_df.columns[:Nens]
    v_grad_columns = chunk_df.columns[Nens:2*Nens]
    remainder = chunk_df.columns[2*Nens:]
    
    chunk_df = pd.concat((chunk_df, OMNI_half_hourly), axis=1)
    
    v_minus_omni = pd.DataFrame(
        chunk_df[v_columns].values - chunk_df['Velocity'].values[:, None],
        columns=[f'v_minus_omni_{i}' for i in range(Nens)],
        index=chunk_df.index
    )
    
    chunk_df = pd.concat((chunk_df, v_minus_omni), axis=1)

    v_minus_omni_columns = v_minus_omni.columns
    
    # Interleave the columns
    interleaved_columns = []
    for a_col, b_col, c_col in zip(v_columns, v_grad_columns, v_minus_omni_columns):
        interleaved_columns.extend([a_col, b_col, c_col])
    interleaved_columns.extend(remainder)
    
    # Reorder the DataFrame using the new column order
    chunk_df_rearranged = chunk_df[interleaved_columns]

    # Append to output file using fastparquet
    fastparquet.write(output_file, chunk_df_rearranged, compression="snappy", append=os.path.exists(output_file))

    # Clear memory
    dfs.clear()
    print(f"Saved batch at CR {cr}")

if not os.path.exists(output_file):
    # Main processing loop
    for i in range(start_cr, end_cr + 1, chunk_size):
        for cr in range(i, min(i + chunk_size, end_cr + 1)):
            file_path = os.path.join(huxt_data_dir, f'HUXt_rotation_{cr}')
            try:
                df = pd.read_parquet(file_path)
                dfs.append(df)
            except:
                print(f'file for CR {cr} not created')
    
        process_chunk()
    
    print('Done')

else:
    print(f'File exists at {output_file}\nDelete old file first')


In [ ]:
%%time

# Check that we have processed correctly

print('File location:', output_file)
df = pd.read_parquet(output_file, engine="fastparquet")

# Check if dataset has unique indices
print('Unique indices:', len(df.index) == len(set(df.index)))

# Columns should read: v_0, v_0_gradient, v_minus_omni_0, v_1, ... , v_99, v_99_gradient, v_minus_omni_99, hpo
print('Data points:', len(df))
df